In [7]:
import numpy as np
from scipy.linalg import norm
import re

In [8]:
H_matrix=1/np.sqrt(2)*np.array([[1, 1],
                                [1,-1]])

CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])

CNOT_tensor=np.reshape(CNOT_matrix, (2,2,2,2))


In [15]:
class Reg: 
    def __init__(self,n):
        self.n=n
        self.psi=np.zeros((2,)*n) 
        self.psi[(0,)*n]=1
        
def H(i,reg): 
    reg.psi=np.tensordot(H_matrix,reg.psi,(1,i)) 
    reg.psi=np.moveaxis(reg.psi,0,i)

def CNOT(control, target, reg):
    reg.psi=np.tensordot(CNOT_tensor, reg.psi, ((2,3),(control, target))) 
    reg.psi=np.moveaxis(reg.psi,(0,1),(control,target))   


In [16]:
def extract_gate_instructions(file_path):
    # Read the text file
    with open(file_path, 'r') as file:
        data = file.read()

    # Extract number of qubits
    num_qubits = int(re.search(r'qubits (\d+)', data).group(1))

    # Extract gate instructions
    gate_instructions = []
    matches = re.findall(r'(\w+)\s+(\d+),?\s?(\d*)', data)
    for match in matches:
        gate = match[0]
        qubits = [int(q) for q in match[1:] if q]
        gate_instructions.append((gate, qubits))

    return num_qubits, gate_instructions[1:]

In [28]:
def execute_gate_instructions(file_path):
    # Extract gate instructions
    num_qubits, gate_instructions = extract_gate_instructions(file_path)

    # Initialize quantum register
    reg = Reg(num_qubits)

    # Execute gate instructions
    for gate, qubits in gate_instructions:
        if gate == 'h':
            H(qubits[0], reg)
        elif gate == 'cx':
            control, target = qubits
            CNOT(control, target, reg)
           

    # print("Final state vector:")
    # print(reg.psi)
    return reg.psi.flatten()

In [29]:
execute_gate_instructions('input.txt')

array([0.70710678, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.70710678])